In [9]:
from gurobipy import *  #引用gurobi求解包
import numpy as np


def Direct_MIP_Return_solver(s,timebound):
    # Model
    OP = Model("DMR")  #定义模型
    
    I = len(s.Ki)
    J = s.number_of_batch
    numberperunit = int(s.number_of_order / s.number_of_batch)
    K = len(s.Ik)
    B = len(s.mb)

    y = OP.addVars(I, J, vtype=GRB.BINARY, name="y")  #定义决策变量
    z = OP.addVars(J, K, vtype=GRB.BINARY, name="z")

    d = []
    for j in range(J):
        d.append([])
        for b in range(B):
            d[j].append(
                OP.addVar(lb=0,
                          ub=s.mb[b],
                          vtype=GRB.CONTINUOUS,
                          name="d[%d,%d]" % (j, b)))

    delta = OP.addVars(J, B, vtype=GRB.BINARY, name="delta")

# The objective is to minimize the costs

    OP.setObjective(
        s.alpha * quicksum(z[j, k] for j in range(J) for k in range(K)) +
        (1 - s.alpha) * (2 * quicksum(d[j][b] for j in range(J) for b in range(B))+s.number_of_batch*len(s.mb)), GRB.MINIMIZE)
    #OP.setObjective(quicksum(delta[j, b] for j in range(J) for b in range(B)), GRB.MINIMIZE) 

    OP.addConstrs(
        sum(y[i, j] for i in range(I)) == numberperunit for j in range(J))
    OP.addConstrs(sum(y[i, j] for j in range(J)) == 1 for i in range(I))
    OP.addConstrs(
        min(numberperunit, len(s.Ik[k])) * z[j, k] >= sum(y[i, j]
                                                          for i in s.Ik[k])
        for j in range(J) for k in range(K))
    OP.addConstrs(
        min(numberperunit, len(s.Ib[b])) * delta[j, b] >= sum(y[i, j]
                                                              for i in s.Ib[b])
        for j in range(J) for b in range(B))
    OP.addConstrs(delta[j, b] <= sum(y[i, j] for i in s.Ib[b])
                  for j in range(J) for b in range(B))
    OP.addConstrs(d[j][b] >= s.rib[i][b] * y[i, j] for j in range(J)
                  for b in range(B) for i in s.Ib[b])

    OP.Params.timelimit = timebound  #设定求解器运行时间上限 单位为秒
    OP.Params.LogToConsole = 0
    FileStr = './Log/' + 'Direct_MIP_Return_solver' + '_' + str(s.number_of_order) + '_' + 'order' + '_' + str(s.number_of_batch) + '_' + 'batch' + '.log'
    OP.Params.LogFile = FileStr

    # Solve

    OP.update()  #更新模型
    OP.optimize()  #开始求解
    OP.setParam("SolutionNumber",0)
    vy=OP.getAttr("xn",y)
    matrix=[[] for j in range(J)]
    for j in range(J):
        matrix[j].extend([i for i,x in enumerate(vy.select('*',j)) if x>0.5])
    s.Direct_MIP_Return_matrix=matrix
    return OP.ObjVal,OP.MIPGap